In [5]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [55]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   zoom_range=[0.5, 1.5],
                                   validation_split=0.2)

valid_datagen = ImageDataGenerator(rescale=1./255, 
                                   validation_split=0.2)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory("../data/processed/train",
                                                 target_size = (64, 64),
                                                 color_mode="grayscale",
                                                 batch_size = 32,
                                                 class_mode = "binary",
                                                 shuffle=True,
                                                 subset="training")

valid_set = train_datagen.flow_from_directory("../data/processed/train",
                                              target_size = (64, 64),
                                              color_mode="grayscale",
                                              batch_size = 32,
                                              class_mode = "binary",
                                              shuffle=False,
                                              subset="validation")

test_set = test_datagen.flow_from_directory("../data/processed/test",
                                            target_size = (64, 64),
                                            color_mode="grayscale",
                                            class_mode = "binary",
                                            batch_size=32)

Found 4000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [30]:
labels_index = { 0 : "mario", 1 : "wario" }
training_set.class_indices

{'mario': 0, 'wario': 1}

In [31]:
# Initialising 
cnn_classifier = Sequential()

# 1st conv. layer
cnn_classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 1), activation = 'relu'))
cnn_classifier.add(MaxPooling2D(pool_size = (2, 2)))

# 2nd conv. layer
cnn_classifier.add(Conv2D(32, (3, 3), activation = 'relu')) #no need to specify the input shape
cnn_classifier.add(MaxPooling2D(pool_size = (2, 2)))

# 3nd conv. layer
cnn_classifier.add(Conv2D(64, (3, 3), activation = 'relu')) #no need to specify the input shape
cnn_classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Flattening
cnn_classifier.add(Flatten())

# Full connection
cnn_classifier.add(Dense(units = 64, activation = 'relu'))
cnn_classifier.add(Dropout(0.5)) # quite aggresive dropout, maybe reduce
cnn_classifier.add(Dense(units = 1, activation = 'sigmoid'))

cnn_classifier.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 31, 31, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 12, 12, 64)        18496     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 6, 6, 64)        

In [35]:
cnn_classifier.compile(optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.001),
                       loss = "binary_crossentropy", 
                       metrics = ["accuracy"])

In [36]:
cnn_classifier.fit(training_set,
                   validation_data=valid_set,
                   epochs = 10)

Epoch 1/10
125/125 [==============================] - 6s 46ms/step - loss: 0.2729 - accuracy: 0.8905 - val_loss: 0.2849 - val_accuracy: 0.8760
Epoch 2/10
125/125 [==============================] - 5s 43ms/step - loss: 0.2676 - accuracy: 0.8898 - val_loss: 0.2504 - val_accuracy: 0.9080
Epoch 3/10
125/125 [==============================] - 5s 39ms/step - loss: 0.2387 - accuracy: 0.9055 - val_loss: 0.2487 - val_accuracy: 0.8830
Epoch 4/10
125/125 [==============================] - 5s 38ms/step - loss: 0.2378 - accuracy: 0.9047 - val_loss: 0.2598 - val_accuracy: 0.8850
Epoch 5/10
125/125 [==============================] - 5s 38ms/step - loss: 0.2210 - accuracy: 0.9120 - val_loss: 0.3876 - val_accuracy: 0.8300
Epoch 6/10
125/125 [==============================] - 5s 38ms/step - loss: 0.2034 - accuracy: 0.9193 - val_loss: 0.2476 - val_accuracy: 0.8970
Epoch 7/10
125/125 [==============================] - 5s 38ms/step - loss: 0.1811 - accuracy: 0.9355 - val_loss: 0.2519 - val_accuracy: 0.8940

In [64]:
cnn_classifier.evaluate(test_set, return_dict=True)

32/32 [==============================] - 1s 27ms/step - loss: 0.6837 - accuracy: 0.7900


{'loss': 0.6837417483329773, 'accuracy': 0.7900000214576721}

In [62]:
32 * 32

1024

In [53]:
test_set